In [2]:
# atualizando o pip para a versão mais atual   
!pip install --upgrade pip

In [1]:
# instalando as depedencias necessárias

!pip install dlib==19.24.6
!pip install --no-cache-dir face_recognition
!pip install tf_keras
!pip install tensorflow
!pip install mediapipe
!pip install opencv-python opencv-python-headless deepface numpy matplotlib fer numpy deepface


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 26.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 1.9/1.9 MB 25.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------------------------------------- 2.6/2.6 MB 18.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/31.2 MB ? eta -:--:--
   -------------- ------------------------- 11.0/31.2 MB 52.9 MB/s eta 0:00:01
   ----------------------------- ---------- 23.1/31.2 MB 56.2 MB/s eta 0:00:01
   ---------------------------------------- 31.2/31.2 MB 50.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/198.5 MB ? eta -:--:--
    --------------------------------------- 3.1/198.5 MB 93.0 MB/s eta 0:00:03
   

  You can safely remove it manually.


In [1]:
import cv2
import random
import json
from deepface import DeepFace
from fer import FER

In [8]:

# Função para gerar um nome aleatório para as pessoas
def generate_person_name(person_id):
    return f"person_{person_id}"

# Função para detectar rostos e emoções
def detect_faces_and_emotions(frame, face_cascade, emotion_detector, person_counter):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    faces_data = []

    for (x, y, w, h) in faces:
        face = frame[y:y + h, x:x + w]
        emotion = emotion_detector.top_emotion(face)
        person_name = generate_person_name(person_counter)
        
        # Salvando o rosto detectado com nome aleatório
        cv2.imwrite(f"faces/{person_name}.jpg", face)
        
        # Armazenar dados da pessoa
        faces_data.append({
            "name": person_name,
            "emotion": emotion[0],  # A emoção detectada
            "coords": (x, y, w, h)
        })
        
        # Desenhando um retângulo ao redor do rosto
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, person_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

    return frame, faces_data

# Função para detectar atividades e anomalias
def detect_anomalies(frame, prev_frame):
    # Lógica para detecção de movimento anômalo (ex: diferença entre frames)
    motion_detected = False
    diff = cv2.absdiff(frame, prev_frame)
    gray_diff = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray_diff, 50, 255, cv2.THRESH_BINARY)
    
    if cv2.countNonZero(thresh) > 500:  # Movimentos significativos
        motion_detected = True

    return motion_detected

# Função para processar o vídeo e gerar relatório
def process_video(input_video):
    cap = cv2.VideoCapture(input_video)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    emotion_detector = FER()
    person_counter = 1
    persons = {}

    prev_frame = None
    frame_number = 0
    report = {"persons": []}

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_number += 1
        # Detecção de rostos e emoções
        frame, faces_data = detect_faces_and_emotions(frame, face_cascade, emotion_detector, person_counter)
        
        # Detecção de anomalias
        if prev_frame is not None:
            motion_detected = detect_anomalies(frame, prev_frame)
            if motion_detected:
                # Marcar anomalia no vídeo
                cv2.putText(frame, 'Anomaly Detected', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Atualizando os dados no relatório
        for person in faces_data:
            name = person["name"]
            if name not in persons:
                persons[name] = {
                    "name": name,
                    "movements": [],
                    "anomalies": [],
                    "emotion": []
                }
            
            # Adicionando movimentos e emoções no relatório
            persons[name]["emotion"].append({
                "frame": frame_number,
                "emotion_name": person["emotion"]
            })
            
            # if motion_detected:
            #     persons[name]["anomalies"].append({
            #         "frame": frame_number,
            #         "anomalie_name": "Movement Anomaly"
            #     })
        
        prev_frame = frame.copy()

        # Exibindo o vídeo com as demarcações
        cv2.imshow('Video', frame)

        # Salvando vídeo com as demarcações
        out.write(frame)

        # Pressione 'q' para sair
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Gerar relatório
    report["persons"] = list(persons.values())
    
    # Salvar relatório em um arquivo JSON
    with open("report.json", "w") as f:
        json.dump(report, f, indent=4)

    cap.release()
    out.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    input_video = "video_analise.mp4"
    # Definindo o arquivo de saída com demarcações
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    out = cv2.VideoWriter('teste.mp4', fourcc, 20.0, (640, 480))
    
    # Processando o vídeo
    process_video(input_video)

KeyboardInterrupt: 